<a href="https://colab.research.google.com/github/harmonydata/experiments/blob/main/Shona_transformer_model_Kufungisisa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Shona symptom questionnaire for the detection of depression and anxiety

https://depts.washington.edu/edgh/zw/hit/web/project-resources/shona_symptom_questionnaire.pdf

Exploring how well Harmony can match Shona model

In [2]:
!pip install transformers harmonydata

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 75.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 45.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [84]:
questions = """Pane pamaimboona muchinyanya kufungisisa kana kufunga zvakawanda here?
Pane pamaimbotadza kuisa pfungwa dzenyu panwechete here?
Maimboshatirwa kanakuita hasha zvenhando here?
Maimborota hope dzinotyisa kana dzisina kunaka here?
Maimboona kana kunzwa zvinhu zvangazvisinga onekwe kana kunzwikwa nevamwe?
Mudumbu menyu maimborwa dza here?
Maimbovhundutswa nezvinhu zvisina mature here?
Maimbota dza kurara kana kushaya hope here?
Pane pamaimbonzwa muchiomerwa neupenyu zvekuti makambochema kana kuti makambonzwa kuda kuchema here?
Maimbonzwa kuneta here?
Pane pamaimboita pfungwa dzekuda kuzviuraya here?
Mainzwa kusafara here mune zvamaiita zuva nezuva?
Basa renyu raive rave kusarira muma shure here?
Mainzwa zvichikuomerai here kuti muzive kuti moita zvipi?"""

In [125]:
official_translation = """Did you sometimes think deeply or think about many things?
Did you find yourself sometimes failing to concentrate?
Did you lose your temper or get annoyed over trivial matters?
Did you have nightmares or bad dreams?
Did you sometimes see or hear things others could not see or hear?
Was your stomach aching?
Were you frightened by trivial things?
Did you sometimes fail to sleep or did you lose sleep?
Were there times when you felt life was so tough you cried or wanted to cry?
Did you feel run down (tired)?
Did you sometimes feel like committing suicide?
Were you generally unhappy with the things you were doing each day?
Was your work lagging behind?
Did you feel you had problems deciding what to do?""".split("\n")

In [87]:
import numpy as np
shona_embeddings = model(questions.split("\n"))

In [88]:
len(shona_embeddings[0][0][0])

768

In [89]:
all_sentence_embeddings = np.asarray([np.mean(token_embeddings[0], axis=0) for token_embeddings in shona_embeddings])

In [90]:
from harmony import example_instruments

In [129]:
example_instruments.keys()

dict_keys(['CES_D English', 'SCARED English', 'GAD-7 Portuguese', 'De Jong Gierveld Loneliness Scale English', 'Market research survey fictional soft drink English', 'GAD-7 English', 'GHQ 12 English', 'Market research survey template English', 'Adult ADHD Self-Report Scale English', 'MacLean Screening Instrument for BPD English', 'RCADS Child Reported English'])

In [142]:
instrument_english = example_instruments['GHQ 12 English']

In [143]:
def get_shona_embedding(texts: list):
  shona_embeddings = model(texts)
  return np.asarray([np.mean(token_embeddings[0], axis=0) for token_embeddings in shona_embeddings])

In [144]:
# import os
# os.environ["HARMONY_NO_PARSING"] = "True"
# os.environ["HARMONY_NO_MATCHING"] = "True"

In [145]:
from harmony import match_instruments_with_function

In [146]:
from harmony.schemas.requests.text import Instrument, Question

In [147]:
shona_instrument = Instrument(questions = [Question(question_text=q, question_no = str(i)) for i, q in enumerate(questions.split("\n"))])

In [148]:
all_questions, similarity, query_similarity, new_vectors_dict = match_instruments_with_function([instrument_english, shona_instrument], None, get_shona_embedding)

In [149]:
threshold = 0.5

In [150]:
pair_to_similarity = dict()
for q1 in range(len(instrument_english.questions)):
    for q2 in range(len(instrument_english.questions), similarity.shape[1]):
        s = similarity[q1][q2]
        pair_to_similarity[(q1, q2)] = s

In [151]:
import operator

In [152]:
seen = set()
for (q1, q2), score in sorted(pair_to_similarity.items(), key=operator.itemgetter(1), reverse=True):
    if q1 in seen or q2 in seen:
      continue
    print (f"{score:.0%}\tGHQ-12 \"{all_questions[q1].question_text}\" matched to Shona: \"{all_questions[q2].question_text}\"")
    print ("\t\ttranslation: ", official_translation[int(all_questions[q2].question_no)])
    seen.add(q1)
    seen.add(q2)

96%	GHQ-12 "Been able to concentrate on what you’re doing? " matched to Shona: "Maimborota hope dzinotyisa kana dzisina kunaka here?"
		translation:  Did you have nightmares or bad dreams?
96%	GHQ-12 "Felt capable of making decisions about things?" matched to Shona: "Pane pamaimboita pfungwa dzekuda kuzviuraya here?"
		translation:  Did you sometimes feel like committing suicide?
96%	GHQ-12 "Been feeling unhappy and depressed?" matched to Shona: "Maimbovhundutswa nezvinhu zvisina mature here?"
		translation:  Were you frightened by trivial things?
96%	GHQ-12 "Felt you couldn’t overcome your difficulties?" matched to Shona: "Pane pamaimbotadza kuisa pfungwa dzenyu panwechete here?"
		translation:  Did you find yourself sometimes failing to concentrate?
96%	GHQ-12 "Been able to enjoy your normal day-to-day activities?" matched to Shona: "Pane pamaimboona muchinyanya kufungisisa kana kufunga zvakawanda here?"
		translation:  Did you sometimes think deeply or think about many things?
96%	G